In [151]:
import pandas as pd
import pyarrow as pa
import json
import lancedb

In [152]:
# 读取 CSV 文件
df = pd.read_csv(r"C:\Lee\work\contract\csv\contract_data.csv")

In [153]:
db = lancedb.connect("C:/Lee/work/db/contract_lancedb")  # 指定数据库目录，不存在会自动创建

In [167]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
embedding_model_id = 'text-embedding-v4'

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_BASE_URL"] = base_url
os.environ["OPENAI_EMBEDDING_MODEL"] = embedding_model_id

In [168]:
from os import name
from lancedb.embeddings import get_registry
# 获取 OpenAI embedding function
openai_embed = get_registry().get("openai").create(name=embedding_model_id,dim = 2048)

In [171]:
schema = pa.schema([
    pa.field("vector", pa.list_(pa.float32(),2048)),
    pa.field("meta_str", pa.utf8()),
    pa.field("doc", pa.utf8()),
    pa.field("contact_no", pa.utf8()),
    pa.field("project_name", pa.utf8()),
    pa.field("subitem_name", pa.utf8()),
    pa.field("date", pa.utf8()),
    pa.field("total_price_str", pa.utf8()),
    pa.field("main_equipments", pa.utf8()),
    pa.field("sub_equipments", pa.utf8()),
    pa.field("supplier", pa.utf8()),
    pa.field("year", pa.int16()), 
    pa.field("total_price", pa.float32()) 
])

In [172]:
from lancedb.pydantic import LanceModel, Vector
class schema(LanceModel):
    meta_str: str = openai_embed.SourceField()  # 用于全文检索
    vector: Vector(openai_embed.ndims()) = openai_embed.VectorField()  # 自动生成向量
    doc: str
    contact_no: str
    project_name: str
    subitem_name: str
    date: str
    total_price_str: str
    main_equipments: str
    sub_equipments: str
    supplier: str
    year: int
    total_price: float

ValueError: Unknown model name text-embedding-v4

In [155]:
table = db.create_table("contract_table", schema=schema,mode="overwrite")

In [156]:
df['vector'] = df['vector'].apply(json.loads)

In [99]:
print(type(df['vector'][0]))
print(df['vector'][0])

<class 'list'>
[0.03252629563212395, -0.03955671563744545, 0.019852474331855774, 0.0008106427267193794, 0.027444060891866684, -0.017840757966041565, 5.926790981902741e-05, 0.03970494866371155, -0.05103409290313721, 0.03216630220413208, 0.02623702958226204, -0.003451683558523655, 0.015987860038876534, -0.00041888721170835197, -0.02706289291381836, 0.0060563283041119576, -0.017089009284973145, -0.07568293064832687, -0.06213030591607094, -0.029222842305898666, 0.03693089634180069, -0.008893908932805061, -0.004939295817166567, 0.023145338520407677, -0.011858545243740082, 0.017819581553339958, -0.027041716501116753, 0.01982071064412594, 0.034749772399663925, -0.023780617862939835, 0.04226724058389664, -0.03229335695505142, -0.01947130635380745, -0.04288134351372719, 0.02511470392346382, 0.020000705495476723, -0.02125008963048458, 0.01353144645690918, 0.03991670906543732, -0.019704243168234825, 0.009126843884587288, -0.007205124944448471, -0.02063598483800888, 3.5837852919939905e-05, -0.0157

In [157]:
df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce').astype('float32')
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('int16') 
# 其它字段转为字符串
str_columns = [
    'meta_str', 'doc', 'contact_no', 'project_name', 'subitem_name',
    'total_price_str', 'supplier', 'main_equipments', 'sub_equipments', 'date'
]
for col in str_columns:
    df[col] = df[col].astype(str)

In [158]:
dict_list = df.to_dict(orient='records')

In [159]:
# table.add(df.to_dict(orient='records'))
table.add(dict_list)

AddResult(version=2)

In [23]:
vector = df.iloc[0]['vector']

In [29]:
type(vector[0])

float

In [58]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
embedding_model_id = 'text-embedding-v4'

In [59]:
def get_embedding(text,model='text-embedding-v4',dimensions=2048):
    client = OpenAI(
        api_key=api_key, 
        base_url=base_url
    )

    completion = client.embeddings.create(
        model=model,
        input=text,
        dimensions=dimensions, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
        encoding_format="float"
    )
    
    return completion.data[0].embedding

In [161]:
embedding = get_embedding('BGZJYR-E-2019-01')

In [164]:
search_results = table.search(embedding,vector_column_name="vector").limit(5).to_pandas()

In [165]:
search_results

,vector,meta_str,doc,contact_no,project_name,subitem_name,date,total_price_str,main_equipments,sub_equipments,supplier,year,total_price,_distance
0,"[0.017127175, -0.036173116, 0.02103012, -0.013...",合同编号: ZJYQJH-E-2019-01B补充协议2 项目名称: 无 子项名称: 无 供...,买方合同编号: ZJYQJH-E-2019-01B补充协议2\n合同类型: None\n项目...,ZJYQJH-E-2019-01B补充协议2,无,无,2021年7月26日,陆万柒仟捌佰元,新增采购型号为500W*230H*500P的活动料斗50套（长、短料斗各25套）,nan,江苏达美输送设备制造有限公司,2021,67800.0,1.242665
1,"[0.036906637, -0.029048426, 0.027203094, 0.044...",合同编号: BGZJYQSCR-E-2019-01 项目名称: 宝钢湛江一、二烧烟气SCR ...,买方合同编号：BGZJYQSCR-E-2019-01 \n合同类型：订货合同 \n项目名...,BGZJYQSCR-E-2019-01,宝钢湛江一、二烧烟气SCR,引风机,2020年3月,伍佰壹拾陆万肆仟圆整,引风机,nan,中国电建集团透平科技有限公司,2020,5164000.0,1.258424
2,"[0.041085847, -0.03659887, 0.038645923, -0.022...",合同编号: BGYR-E-2015-01 项目名称: 宝山钢铁股份有限公司炼铁厂三烧大修改造...,买方合同编号：BGYR-E-2015-01 \n合同类型：供货合同 \n项目名称：宝山钢...,BGYR-E-2015-01,宝山钢铁股份有限公司炼铁厂三烧大修改造工程环冷余热锅炉EP,EP1X（80+20）t/h环冷余热锅炉设备供货,2015年9月24日,人民币捌佰肆拾柒万圆整,nan,"锅炉本体,激波吹灰装置,仪表、阀门,易损易耗,专用工具,运输保险费,电动葫芦、卸灰阀,技术服务费",江苏东九重工股份有限公司,2015,8470000.0,1.272074
3,"[0.024459464, -0.019751698, 0.048626002, -0.01...",合同编号: BGFS-2019-01 项目名称: 宝钢焚烧炉出灰渣系统技改项目 子项名称: ...,买方合同编号：BGFS-2019-01\n 合同类型：订货合同\n 项目名称：宝...,BGFS-2019-01,宝钢焚烧炉出灰渣系统技改项目,冷却输送成套设备,2019年6月5日,肆拾伍万伍仟元整,冷却输送成套系统,"焚烧炉出渣口格栅网,DN300星型卸料阀,DG-03水冷滚筒式冷渣机（成套）,B500电磁除...",江苏荣达电力设备有限公司,2019,455000.0,1.327889
4,"[0.032526296, -0.039556716, 0.019852474, 0.000...",合同编号: BGZJQT-E-2019-01 项目名称: 宝钢湛江钢铁有限公司炼铁厂球团烟气...,买方合同编号：BGZJQT-E-2019-01 \n合同类型：订货合同 \n项目名称：宝...,BGZJQT-E-2019-01,宝钢湛江钢铁有限公司炼铁厂球团烟气脱硝设施球团烟气SCR脱硝系统,回转式烟气换热器（GGH）,2019年10月,肆佰玖拾玖万 圆整,回转式烟气换热器（GGH）,nan,容客傲华新能源科技（上海）有限公司,2019,4990000.0,1.340226


In [140]:
from lancedb.query import MatchQuery

In [149]:
query_text = "BGZJYR-E-2019-01"
# Filters without vector search
filtered_result = (
    table.search(embedding)
    .where(f"contact_no='{query_text}'")
    .limit(5)
    .to_pandas()
)

In [150]:
filtered_result

,vector,meta_str,doc,contact_no,project_name,subitem_name,date,total_price_str,main_equipments,sub_equipments,supplier,year,total_price,_distance
0,"[0.036996108, -0.029621813, 0.029372541, 0.007...",合同编号: BGZJYR-E-2019-01 项目名称: 宝钢湛江三高炉烧结余热发电 子项名...,买方合同编号: BGZJYR-E-2019-01 \n合同类型: 订货合同 \n项目名称...,BGZJYR-E-2019-01,宝钢湛江三高炉烧结余热发电,循环风机,2019年11月,贰佰伍拾捌万圆整,循环风机,"差动式调节门,进气箱（含轴封）,机壳装配,集流器,电机底座(带调整螺丝),执行器配件,叶轮,...",金通灵科技集团股份有限公司,2019,2580000.0,1.359221


In [160]:
table.to_pandas()

,vector,meta_str,doc,contact_no,project_name,subitem_name,date,total_price_str,main_equipments,sub_equipments,supplier,year,total_price
0,"[0.032526296, -0.039556716, 0.019852474, 0.000...",合同编号: BGZJQT-E-2019-01 项目名称: 宝钢湛江钢铁有限公司炼铁厂球团烟气...,买方合同编号：BGZJQT-E-2019-01 \n合同类型：订货合同 \n项目名称：宝...,BGZJQT-E-2019-01,宝钢湛江钢铁有限公司炼铁厂球团烟气脱硝设施球团烟气SCR脱硝系统,回转式烟气换热器（GGH）,2019年10月,肆佰玖拾玖万 圆整,回转式烟气换热器（GGH）,nan,容客傲华新能源科技（上海）有限公司,2019,4990000.0
1,"[0.022459952, -0.032365043, 0.04580316, -0.026...",合同编号: BGYR3-E-2024-01 项目名称: 宝山钢铁股份有限公司宝山基地炼铁厂三...,买方合同编号：BGYR3-E-2024-01 \n合同类型：订货合同 \n项目名称：宝山...,BGYR3-E-2024-01,宝山钢铁股份有限公司宝山基地炼铁厂三烧环冷机升级改造项目余热锅炉改造设备,大烟道余热锅炉,2024年11月,1500000.00元,大烟道余热锅炉,"锅炉本体,蒸发器,省煤器,汽包,汽水管道,弯头,钢制组件,安全阀、仪表,油漆,指导安装费,包...",江苏中天能源设备有限公司,2024,1500000.0
2,"[0.0059464676, -0.017559974, 0.03662985, 0.012...",合同编号: AGZSFS2023-01 项目名称: 安钢炼铁厂烧结机头烟气脱硫脱硝系统制酸废...,买方合同编号：AGZSFS2023-01 \n合同类型：订货合同 \n项目名称：安钢炼铁...,AGZSFS2023-01,安钢炼铁厂烧结机头烟气脱硫脱硝系统制酸废水处理项目,压滤机备件,2023年5月,￥32376.00元,"三级板框压滤机A,三级板框压滤机B,废水板框压滤机整套滤布,气动球阀,多级离心泵,压榨胶管,...","三级板框压滤机A,三级板框压滤机B,废水板框压滤机整套滤布,气动球阀,多级离心泵,压榨胶管,...",景津装备股份有限公司,2023,32376.0
3,"[0.0156096, -0.023960331, 0.03227062, 0.008274...",合同编号: AYGTYQJH-2017-01 项目名称: 安阳钢铁烧结机烟气脱硫脱硝项目 子...,买方合同编号：AYGTYQJH-2017-01\n合同类型：订货合同\n项目名称：安阳钢铁烧...,AYGTYQJH-2017-01,安阳钢铁烧结机烟气脱硫脱硝项目,增压风机,2017年7月14日,540万元,"1#烧结机脱硫增压风机,2#烧结机脱硫增压风机10KV 4200kw 8P 变频,3#烧结机...","1#烧结机脱硫增压风机,2#烧结机脱硫增压风机10KV 4200kw 8P 变频,3#烧结机...",上海电气鼓风机厂有限公司,2017,540.0
4,"[0.013421798, -0.028928123, 0.013315445, 0.021...",合同编号: BGZJZS-E-2020-01 项目名称: 宝钢湛江SCR、CO催化氧化中试 ...,买方合同编号：BGZJZS-E-2020-01\n合同类型：订货合同\n项目名称：宝钢湛江S...,BGZJZS-E-2020-01,宝钢湛江SCR、CO催化氧化中试,引风机,2020年3月,壹拾壹万元整,引风机,"引风机,配套变频电机（安徽皖南）,电机底座,进出口软连接,电动执行器380V",鞍山钢峰风机有限责任公司,2020,110000.0
5,"[0.036906637, -0.029048426, 0.027203094, 0.044...",合同编号: BGZJYQSCR-E-2019-01 项目名称: 宝钢湛江一、二烧烟气SCR ...,买方合同编号：BGZJYQSCR-E-2019-01 \n合同类型：订货合同 \n项目名...,BGZJYQSCR-E-2019-01,宝钢湛江一、二烧烟气SCR,引风机,2020年3月,伍佰壹拾陆万肆仟圆整,引风机,nan,中国电建集团透平科技有限公司,2020,5164000.0
6,"[0.030830877, -0.024685118, 0.036547862, 0.003...",合同编号: BGDSYR-2020-01 项目名称: 宝钢德盛不锈钢有限公司烧结余热项目 子...,买方合同编号: BGDSYR-2020-01 \n合同类型: 订货合同 \n项目名称: ...,BGDSYR-2020-01,宝钢德盛不锈钢有限公司烧结余热项目,循环风机,2020年10月,壹佰肆拾捌万元整,循环风机,nan,金通灵科技集团股份有限公司,2020,1480000.0
7,"[0.036996108, -0.029621813, 0.029372541, 0.007...",合同编号: BGZJYR-E-2019-01 项目名称: 宝钢湛江三高炉烧结余热发电 子项名...,买方合同编号: BGZJYR-E-2019-01 \n合同类型: 订货合同 \n项目名称...,BGZJYR-E-2019-01,宝钢湛江三高炉烧结余热发电,循环风机,2019年11月,贰佰伍拾捌万圆整,循环风机,"差动式调节门,进气箱（含轴封）,机壳装配,集流器,电机底座(带调整螺丝),执行器配件,叶轮,...",金通灵科技集团股份有限公司,2019,2580000.0
8,"[0.0351361, -0.016792988, 0.020978317, -0.0086...",合同编号: BGYES-E-2017-01 项目名称: 宝山钢铁股份有限公司炼铁厂一二烧结整...,买方合同编号：BGYES-E-2017-01\n 合同类型：订货合同\n 项目名...,BGYES-E-2017-01,宝山钢铁股份有限公司炼铁厂一二烧结整合大修改造工程环冷余热锅炉EP,风机,2017年12月27日,贰佰肆拾叁万 圆整,循环风机 RJ55-DW3400F,nan,江苏金通灵流体机械科技股份有限公司,2017,2430000.0
9,"[0.0016886148, -0.04260427, 0.04596883, -0.014...",合同编号: CIE2020-AGTLTX-JAHT01 项目名称: 中冶长天安钢烧结机机头烟...,买方合同编号\n CIE2020-AGTLTX-JAHT01\n 合同类型\n ...,CIE2020-AGTLTX-JAHT01,中冶长天安钢烧结机机头烟气深度治理二期项目桩基工程,无,2020年04月,伍佰贰拾玖万叁仟捌佰陆拾叁元整,nan,nan,安阳市钢城建筑有限公司,2020,5293863.0
